**Imports**

In [ ]:
import os
import sys
import glob
import time
import yaml as yl
import pickle as pk
import numpy as np
import pandas as pd
import subprocess
import datetime
import matplotlib.pyplot as plt

from google.colab import drive
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, classification_report

**Stellar** **Graph**

In [ ]:
%pip install -q stellargraph==1.2.1

import stellargraph as sg

from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarDiGraph

**Tensorflow**

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import categorical_crossentropy

**Profiler**

In [ ]:
#%pip install line_profiler
%pip install memory_profiler
%pip install wandb

#import line_profiler
%load_ext memory_profiler
#import wandb

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


**Functions**

In [ ]:
def execute(cmdline):
  """Execute a command."""
  try:
    ret = subprocess.run(cmdline,
                        shell=True,
                        check=True,
                        capture_output=True)
  except subprocess.CalledProcessError:
    print('Execute: {}'.format(cmdline))
    return ''

  return ret.stdout.decode(), ret.stderr.decode()

def save(filename, data):
  """Save a file."""
  fout = open(filename, 'w')
  fout.write(data)
  fout.close()

def get_edges_dataFrame(edges, edge_type='original'):
  """Return the edges."""
  source = []
  target = []
  type_ = []
  for node1, node2, data in edges:
      source.append(node1)
      target.append(node2)
      if edge_type == 'original':
          type_.append(data)
      elif edge_type == 'default':
          type_.append('default')
      elif edge_type == 'random':
          type_.append(np.random.random())
      else:
          print('Error: edge_type', edge_type)
          sys.exit(1)

  return pd.DataFrame({'source': source, 'target': target, 'type': type_})

def stellar_graph_no_edge_features(graph):
  """Remove the type of the edges: <type> -> default."""

  edges = get_edges_dataFrame(graph.edges(1), 'default')

  s_graph = StellarDiGraph(graph.node_features(),
                           edges=edges,
                           edge_type_column="type")
  return s_graph

def stellar_graph_no_node_features(graph):
  """Remove the node features."""

  edges = get_edges_dataFrame(graph.edges(1), 'original')

  nof_nodes = graph.number_of_nodes()
  nof_features = graph.node_feature_shapes()['default'][0]

  node_features = np.zeros((nof_nodes, nof_features))

  s_graph = StellarDiGraph(node_features,
                           edges=edges,
                           edge_type_column="type")
  return s_graph

def stellar_graph_random_edge_features(graph):
  """Add random edge features."""

  edges = get_edges_dataFrame(graph.edges(1), 'random')

  s_graph = StellarDiGraph(graph.node_features(),
                           edges=edges,
                           edge_type_column="type")
  return s_graph

def stellar_graph_random_node_features(graph):
  """Add random node features."""

  edges = get_edges_dataFrame(graph.edges(1), 'original')

  nof_nodes = graph.number_of_nodes()
  nof_features = graph.node_feature_shapes()['default'][0]

  node_features = np.random.random((nof_nodes, nof_features))

  s_graph = StellarDiGraph(node_features,
                           edges=edges,
                           edge_type_column="type")
  return s_graph

def stellar_graph_no_features(graph):
  """Remove the egde/node features."""

  edges = get_edges_dataFrame(graph.edges(1), 'default')

  nof_nodes = graph.number_of_nodes()
  nof_features = graph.node_feature_shapes()['default'][0]

  node_features = np.zeros((nof_nodes, nof_features))

  s_graph = StellarDiGraph(node_features,
                           edges=edges,
                           edge_type_column="type")
  return s_graph

def create_model(graphs, classes, model_type='1_gcnn_32_v1'):
  print('Generator ...')
  %memit generator = PaddedGraphGenerator(graphs=graphs)

  if model_type == '1_gcnn_32_v1':
    k = 35  # the number of rows for the output tensor
    layer_sizes = [32]

    dgcnn_model = DeepGraphCNN(
      layer_sizes=layer_sizes,
      activations=["relu"],
      k=k,
      bias=False,
      generator=generator,  
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    # Add the convolutional, max pooling, and dense layers
    x_out = Conv1D(filters=16, kernel_size=3, activation="relu")(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=5, activation="relu")(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)
    outputs = Dense(units=classes, activation="sigmoid")(x_out)

    #
    # Create the model and prepare it for training by specifying the loss and optimization algorithm
    #
    model = Model(inputs=x_inp, outputs=outputs)
  elif model_type == '1_gcnn_32_v2':
    k = 35  # the number of rows for the output tensor
    layer_sizes = [32]

    dgcnn_model = DeepGraphCNN(
      layer_sizes=layer_sizes,
      activations=["relu"],
      k=k,
      bias=False,
      generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    # Add the convolutional, max pooling, and dense layers
    x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)
    outputs = Dense(units=classes, activation="sigmoid")(x_out)
    
    #
    # Create the model and prepare it for training by specifying the loss and optimization algorithm
    #
    model = Model(inputs=x_inp, outputs=outputs) 
  elif model_type == '1_gcnn_256_v1':
    k = 35  # the number of rows for the output tensor
    layer_sizes = [256]

    dgcnn_model = DeepGraphCNN(
      layer_sizes=layer_sizes,
      activations=["relu"],
      k=k,
      bias=False,
      generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    # Add the convolutional, max pooling, and dense layers
    x_out = Conv1D(filters=16, kernel_size=3, activation="relu")(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=5, activation="relu")(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)
    outputs = Dense(units=classes, activation="sigmoid")(x_out)
    
    #
    # Create the model and prepare it for training by specifying the loss and optimization algorithm
    #
    model = Model(inputs=x_inp, outputs=outputs)  
  elif model_type == '1_gcnn_256_v2':
    k = 35  # the number of rows for the output tensor
    layer_sizes = [256]

    dgcnn_model = DeepGraphCNN(
      layer_sizes=layer_sizes,
      activations=["relu"],
      k=k,
      bias=False,
      generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    # Add the convolutional, max pooling, and dense layers
    x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)
    outputs = Dense(units=classes, activation="sigmoid")(x_out)
    
    #
    # Create the model and prepare it for training by specifying the loss and optimization algorithm
    #
    model = Model(inputs=x_inp, outputs=outputs)           
  elif model_type == '4_gcnn_v1':
    k = 35  # the number of rows for the output tensor
    layer_sizes = [256, 256, 256, classes]

    dgcnn_model = DeepGraphCNN(
      layer_sizes=layer_sizes,
      activations=["relu", "relu", "relu", "relu"],
      k=k,
      bias=False,
      generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    # Add the convolutional, max pooling, and dense layers
    x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)
    outputs = Dense(units=classes, activation="sigmoid")(x_out)
    
    #
    # Create the model and prepare it for training by specifying the loss and optimization algorithm
    #
    model = Model(inputs=x_inp, outputs=outputs) 

  return model

**Arguments**

In [ ]:
# Google Driver
FLAGS_driver = '/content/drive'

# Train data ratio
FLAGS_train_ratio = 0.75

# Validation data ratio
FLAGS_val_ratio = 0.25

# Test data ratio
FLAGS_test_ratio = 0.20

# Number of epochs
FLAGS_epochs = 100

# Verbose
FLAGS_training_verbose = 1

# Store the results?
FLAGS_store_results = True

# Store the model?
FLAGS_store_model = False

# Store Wandb data?
FLAGS_store_wandb = False

# Wandb project name
FLAGS_project_name = 'bestseqssblp_200'

# Breakdown the runtime
FLAGS_times = {}

# Dataset
#
# classifyapp/Oz
# classifyapp/Oz
# bestseqssblp/O0/group1.1000
#
FLAGS_dataset = 'bestseqssblp2021.poj/200/Oz'

# Result directory
#
# classifyapp/POJ-104
FLAGS_r_d_name = 'bestseqssblp_200'

# Result directory suffix
FLAGS_suffix = ''

# Model
FLAGS_model = '1_gcnn_32_v1'

# Source ratio (use all data?)
#
# It is not possible to use all POJ dataset in Colab (error -> out of memory).
# It is possible to load 450 sources (90%) per group.
#
# POJ ratios -> 0.9, 0.45, 0.09
FLAGS_source_ratio = 1.0

# Filter the source data
FLAGS_filter_source = False

# Number of labels
#
# POJ labels -> 104, 52, 10
FLAGS_labels = 11

# Features type: all, no edges, no nodes
FLAGS_features = 'no_features'

# Round
FLAGS_round = 'round3'

# Random state
FLAGS_random_state = {'round1': 123456789, 'round2': 12345678, 'round3': 1234567}

# Graph
#
# programlnoroot
# cfgcallnoroot
# cfgcallcompactnoroot
# cfgcallcompact1enoroot
# cdfgcallnoroot
# cdfgcallcompactnoroot
# cdfgcallcompact1enoroot
# cdfgplusnoroot
FLAGS_graph = 'programlnoroot'

# Embeddings
#
# bag_of_words
# inst2vec
# ir2vec
# opcodes
FLAGS_embeddings = 'inst2vec' 

# Results directory
last = '{}_{}_{}'.format(FLAGS_features, FLAGS_labels, FLAGS_source_ratio) if not FLAGS_suffix else '{}_{}_{}_{}'.format(FLAGS_features, FLAGS_labels, FLAGS_source_ratio, FLAGS_suffix)
FLAGS_results_directory = os.path.join(FLAGS_driver,
                                        'My Drive/classification/{}'.format(FLAGS_r_d_name),
                                        FLAGS_round,
                                        '{}/{}'.format(FLAGS_graph, FLAGS_embeddings),
                                        last)

# Dataset directory
FLAGS_dataset_directory = os.path.join(FLAGS_graph,
                                        FLAGS_embeddings)

**Import the dataset**

In [ ]:
#
# Cleanup
#
command = 'rm -rf {}* gpu.log gpu_usage.sh graphs-number-of-nodes.yaml'.format(FLAGS_graph)
!$command

#
# The graphs
#
wget = 'wget www.csl.uem.br/repository/data/{}/{}.tar.xz'.format(FLAGS_dataset, FLAGS_graph)
tar = 'tar xfJ {}.tar.xz'.format(FLAGS_graph)
!$wget
!$tar

if FLAGS_filter_source:
  # 
  # Graphs' number of nodes (to filter data)
  #
  wget = 'wget www.csl.uem.br/repository/data/{}/graphs-number-of-nodes.yaml'.format(FLAGS_dataset)
  !$wget


--2021-08-23 20:19:45--  http://www.csl.uem.br/repository/data/bestseqssblp2021.poj/200/Oz/programlnoroot.tar.xz
Resolving www.csl.uem.br (www.csl.uem.br)... 186.233.153.209
Connecting to www.csl.uem.br (www.csl.uem.br)|186.233.153.209|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.csl.uem.br/repository/data/bestseqssblp2021.poj/200/Oz/programlnoroot.tar.xz [following]
--2021-08-23 20:19:46--  https://www.csl.uem.br/repository/data/bestseqssblp2021.poj/200/Oz/programlnoroot.tar.xz
Connecting to www.csl.uem.br (www.csl.uem.br)|186.233.153.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10870744 (10M) [application/octet-stream]
Saving to: ‘programlnoroot.tar.xz’

programlnoroot.tar. 100%[===================>]  10.37M  2.38MB/s    in 5.1s    

2021-08-23 20:19:52 (2.04 MB/s) - ‘programlnoroot.tar.xz’ saved [10870744/10870744]



**Background Monitor**

In [ ]:
!rm -f gpu.log gpu_usage.sh

In [ ]:
%%writefile gpu_usage.sh
#! /bin/bash
#comment: run for 24 hours, change it as per your use
end=$((SECONDS+86400))

while [ $SECONDS -lt $end ]; do
    nvidia-smi --format=csv --query-gpu=power.draw,utilization.gpu,memory.used,memory.free,fan.speed,temperature.gpu >> gpu.log
    #comment: or use below command and comment above using #
    #nvidia-smi dmon -i 0 -s mu -d 1 -o TD >> gpu.log
done

Writing gpu_usage.sh


In [ ]:
%%bash --bg

bash gpu_usage.sh

Starting job # 10 in a separate thread.


**Open Google Drive**

In [ ]:
if FLAGS_store_results:
  drive.mount('/content/drive')

Mounted at /content/drive


**Initiate Wandb**

In [ ]:
if FLAGS_store_wandb:
  PROJECT_NAME = '{}_{}_{}_{}_{}'.format(FLAGS_project_name,
                                        FLAGS_round,
                                        FLAGS_graph,
                                        FLAGS_embeddings,
                                        '{}_{}_{}'.format(FLAGS_features,
                                                          FLAGS_labels,
                                                          FLAGS_source_ratio)
                                            )
  wandb.init(project=PROJECT_NAME, entity='andersonfaustino')

**Load the dataset statistics**

In [ ]:
if FLAGS_filter_source:
  fin = open('graphs-number-of-nodes.yaml', 'r')
  graphs_size = yl.load(fin)
  fin.close()

**Load the dataset**

In [ ]:
labels = []
graphs = [] 

start = time.time()

if FLAGS_filter_source:

  for label in range(1, FLAGS_labels+1):
    sources = graphs_size[label]
    sources.sort()        
    
    nof_sources = int(len(sources) * FLAGS_source_ratio)  

    for i, source in enumerate(sources):
      try:
        filename = '{}/{}/{}.pk'.format(FLAGS_dataset_directory, label, source[1])
        fin = open(filename, 'rb')
        graph = pk.load(fin)
        fin.close()
      except:
        print('Erro load', FLAGS_dataset_directory, label, source[1], flush=True)
        continue

      if FLAGS_features == 'no_edge_features':
        graph = stellar_graph_no_edge_features(graph)
      elif FLAGS_features == 'no_node_features':
        graph = stellar_graph_no_node_features(graph)
      elif FLAGS_features == 'random_edge_features':
        graph = stellar_graph_random_edge_features(graph)        
      elif FLAGS_features == 'random_node_features':
        graph = stellar_graph_random_node_features(graph)       
      elif FLAGS_features == 'no_features':
        graph = stellar_graph_no_features(graph)

      labels.append(label-1)
      graphs.append(graph)      

      if i == nof_sources - 1:
        break
else:

  for label in range(1, FLAGS_labels+1):
    sources = glob.glob('{}/{}/*.pk'.format(FLAGS_dataset_directory, label))          
    
    nof_sources = int(len(sources) * FLAGS_source_ratio)  

    for i, source in enumerate(sources):
      try:
        fin = open(source, 'rb')
        graph = pk.load(fin)
        fin.close()
      except:
        print('Erro load', source, flush=True)
        continue

      if FLAGS_features == 'no_edge_features':
        graph = stellar_graph_no_edge_features(graph)
      elif FLAGS_features == 'no_node_features':
        graph = stellar_graph_no_node_features(graph)
      elif FLAGS_features == 'random_edge_features':
        graph = stellar_graph_random_edge_features(graph)         
      elif FLAGS_features == 'random_node_features':
        graph = stellar_graph_random_node_features(graph)         
      elif FLAGS_features == 'no_features':
        graph = stellar_graph_no_features(graph)

      labels.append(label-1)
      graphs.append(graph)      

      if i == nof_sources - 1:
        break

end = time.time()

FLAGS_times['loading'] = end - start



**Encode the class values**

In [ ]:
graphs_label = pd.Series(labels, name='label', dtype="category")  

summary = pd.DataFrame(
  [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
  columns=['nodes', 'edges'],
)
print('\n', summary.describe().round(1), flush=True)
print('\n', graphs_label.value_counts().to_frame(), flush=True)

graphs_label = pd.get_dummies(graphs_label)
classes = graphs_label.shape[1]


         nodes   edges
count  2200.0  2200.0
mean    177.0   297.1
std      91.8   168.8
min      10.0     9.0
25%     124.0   206.0
50%     153.0   255.0
75%     217.0   364.0
max    1584.0  2904.0

     label
10    200
9     200
8     200
7     200
6     200
5     200
4     200
3     200
2     200
1     200
0     200


**Create the model**

In [ ]:
model = create_model(graphs, classes, FLAGS_model)
model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics=['accuracy'])
model.summary()

Generator ...
peak memory: 11113.31 MiB, increment: 0.02 MiB
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           [(None, None, 200)]  0                                            
__________________________________________________________________________________________________
dropout_18 (Dropout)            (None, None, 200)    0           input_28[0][0]                   
__________________________________________________________________________________________________
input_30 (InputLayer)           [(None, None, None)] 0                                            
__________________________________________________________________________________________________
graph_convolution_9 (GraphConvo (None, None, 32)     6400        dropout_18[0][0]                 
                               

**Prepare the dataaset**

In [ ]:
train_graphs, test_graphs = model_selection.train_test_split(
  graphs_label,
  train_size=1.0-FLAGS_test_ratio,
  test_size=FLAGS_test_ratio,
  stratify=graphs_label,
  random_state=FLAGS_random_state[FLAGS_round]
)

train_graphs, val_graphs = model_selection.train_test_split(
  train_graphs,
  train_size=FLAGS_train_ratio,
  test_size=FLAGS_val_ratio,
  stratify=train_graphs,
  random_state=FLAGS_random_state[FLAGS_round]
)

%memit gen = PaddedGraphGenerator(graphs=graphs)
%memit train_gen = gen.flow(list(train_graphs.index), targets=train_graphs.values, batch_size=50, symmetric_normalization=False)
%memit val_gen = gen.flow(list(val_graphs.index), targets=val_graphs.values, batch_size=1, symmetric_normalization=False)
%memit test_gen = gen.flow(list(test_graphs.index), targets=test_graphs.values, batch_size=1, symmetric_normalization=False) 

print('Training:', train_graphs.shape[0], flush=True)
print('Validation:', val_graphs.shape[0], flush=True)
print('Test:', test_graphs.shape[0], flush=True)

peak memory: 11113.32 MiB, increment: 0.00 MiB
peak memory: 11113.57 MiB, increment: 0.25 MiB
peak memory: 11115.16 MiB, increment: 1.60 MiB
peak memory: 11116.89 MiB, increment: 1.72 MiB
Training: 1320
Validation: 440
Test: 440


**Training**

In [ ]:
start = time.time()
%memit history = model.fit(train_gen, epochs=FLAGS_epochs, verbose=FLAGS_training_verbose, validation_data=val_gen, shuffle=True)
end = time.time()

FLAGS_times['training'] = end - start

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_9/sort_pooling_9/map/while/gradients/model_9/sort_pooling_9/map/while/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_9/sort_pooling_9/map/while/gradients/model_9/sort_pooling_9/map/while/GatherV2_grad/Reshape:0", shape=(None, None), dtype=float32), dense_shape=Tensor("gradient_tape/model_9/sort_pooling_9/map/while/gradients/model_9/sort_pooling_9/map/while/GatherV2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


27/27 [==============================] - 10s 307ms/step - loss: 2.3979 - accuracy: 0.0833 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 2/100
27/27 [==============================] - 6s 206ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 3/100
27/27 [==============================] - 6s 240ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 4/100
27/27 [==============================] - 6s 240ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 5/100
27/27 [==============================] - 6s 236ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 6/100
27/27 [==============================] - 6s 242ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 7/100
27/27 [==============================] - 6s 234ms/step - loss: 2.3979 - accuracy: 0.0909 - val_loss: 2.3979 - val_accuracy: 0.0909
Epoch 8/10

**Testing**

In [ ]:
start = time.time()
%memit test_metrics = model.evaluate(test_gen)
end = time.time()

FLAGS_times['evaluating'] = end - start

test_metrics_dict = {}
for name, val in zip(model.metrics_names, test_metrics):
  print('{}: {:0.4f}'.format(name, val), flush=True)
  test_metrics_dict[name] = val

440/440 [==============================] - 2s 5ms/step - loss: 2.3979 - accuracy: 0.0909
peak memory: 11898.82 MiB, increment: 0.09 MiB
loss: 2.3979
accuracy: 0.0909


**Predicting**

In [ ]:
start = time.time()
%memit predicted = model.predict(test_gen)
end = time.time()

FLAGS_times['predicting'] = end - start

peak memory: 11899.31 MiB, increment: 0.59 MiB


**Statistics**

In [ ]:
pred_y = predicted.argmax(axis=-1)

cm = confusion_matrix(test_graphs.idxmax(axis=1), pred_y)
print('Confusion matrix')
print(cm)

cr = classification_report(test_graphs.idxmax(axis=1), pred_y)
print('\n\nClassification report')
print(cr)

Confusion matrix
[[ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]]


Classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           1       0.00      0.00      0.00        40
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        40
           4       0.00      0.00      0.00        40
           5       0.00      0.00      0.00        40
           6       0.00      0.00      0.00        40
           7       0.00      0.00      0.00        40
           8       0.09      1.00      0.17        40
           9       0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Store the results**

In [ ]:
if FLAGS_store_results:
  #
  # Create the output directory
  #
  os.makedirs(FLAGS_results_directory, exist_ok=True)

  #
  # Store the results
  #

  # History
  fout = open('{}/history.yaml'.format(FLAGS_results_directory), 'w')
  yl.dump(history.history, fout)
  fout.close()

  # Summary
  fout = open('{}/summary.yaml'.format(FLAGS_results_directory), 'w')
  yl.dump(summary.describe().to_dict(), fout)
  fout.close()

  # Test metrics
  fout = open('{}/test_metrics.yaml'.format(FLAGS_results_directory), 'w')
  yl.dump(test_metrics_dict, fout)
  fout.close()

  # Graphs
  train_graphs.to_pickle('{}/train_graphs.pkl'.format(FLAGS_results_directory))
  val_graphs.to_pickle('{}/val_graphs.pkl'.format(FLAGS_results_directory))
  test_graphs.to_pickle('{}/test_graphs.pkl'.format(FLAGS_results_directory))

  # Predicted
  np.savez_compressed('{}/predicted'.format(FLAGS_results_directory), values=predicted)

  # Confusion matrix
  np.savez_compressed('{}/confusion_matrix'.format(FLAGS_results_directory), values=cm)

  # Classification report
  fout = open('{}/classification_report.pk'.format(FLAGS_results_directory), 'wb')
  pk.dump(cr, fout)
  fout.close()

  # Time
  fout = open('{}/elapsed_time.yaml'.format(FLAGS_results_directory), 'w')
  yl.dump(FLAGS_times, fout)
  fout.close()

  # gpu.log
  command = 'cp gpu.log "{}"'.format(FLAGS_results_directory)
  !$command

  #
  # Store the hadware specifications
  #
  spec, _ = execute("nvidia-smi")
  filename = '{}/nvidia-smi.txt'.format(FLAGS_results_directory)
  save(filename, spec)

  spec, _ = execute("cat /proc/cpuinfo")
  filename = '{}/cpuinfo.txt'.format(FLAGS_results_directory)
  save(filename, spec)

  spec, _ = execute("cat /proc/meminfo")
  filename = '{}/meminfo.txt'.format(FLAGS_results_directory)
  save(filename, spec)

  #
  # Store the model
  #
  if FLAGS_store_model:
    directory = '{}/model'.format(FLAGS_results_directory)
    os.makedirs(directory, exist_ok=True)
    model.save(directory)


**Free the memory**

In [ ]:
del labels
del graphs
del graphs_label
del generator
del model
del train_graphs
del test_graphs
del val_graphs
del gen
del train_gen
del val_gen
del test_gen
del history
del test_metrics
del test_metrics_dict
del predicted
del cm
del cr

**Flush Google Driver**

In [ ]:
if FLAGS_store_results:
  #
  # Flush the Driver
  #
  drive.flush_and_unmount()

**Remove the dataset**

In [ ]:
# Kill gpu_usage
!pid=`ps -aux |grep gpu_usage.sh | awk '{print $2}' | head -n 1` ; kill -9 $pid

# Clean
command = 'rm -rf {}* gpu.log gpu_usage.sh graphs-number-of-nodes.yaml'.format(FLAGS_graph)
!$command

**Finish**

In [ ]:
for key, value in FLAGS_times.items():
    print(key, value)

loading 6.627687692642212
training 641.4394586086273
evaluating 2.9691309928894043
predicting 2.5142109394073486
